# PROJETO EDA - ANÁLISE DE SENTIMENTOS CYBERPUNK 2077

## Disciplina: IMD1151 - Ciência de Dados

### PROBLEMAS:

- Como a percepção dos jogadores sobre Cyberpunk 2077 evoluiu ao longo
do tempo (desde o lançamento problemático em 2020 até 2025)?
- Quais
diferenças existem entre as comunidades de língua inglesa e portuguesa
em termos de sentimento e aspectos mais valorizados/criticados?

### OBJETIVOS:
1. Analisar evolução temporal do sentimento das reviews
2. Comparar percepções entre comunidades EN x PT-BR
3. Relacionar tempo jogado com sentimento da review

### DATASET:
- Fonte: Steam Reviews API (Cyberpunk 2077 - App ID 1091500)
- Idiomas: Inglês (en) e Português brasileiro (brazilian)
- Período: 10/12/2020 (lançamento) até 14/10/2025
- Total coletado: 41,427 reviews
- Comentários pós tratamento: 41,409


**Esse notebook é responsável por uma limpeza e tratamento dos dados, bem como, uma visão geral dos dados obtidos para tomadas de decisões**



## CARREGAMENTO DAS BIBLIOTECAS E DA BASE DE DADOS

In [24]:
import pandas as pd
import ast
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

In [25]:
df = pd.read_csv('/content/steam_reviews_cyberpunk_br_en.csv')
df

/tmp/ipython-input-2893359385.py:1: DtypeWarning: Columns (16) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/content/steam_reviews_cyberpunk_br_en.csv')


,recommendationid,author,language,review,timestamp_created,timestamp_updated,voted_up,votes_up,votes_funny,weighted_vote_score,comment_count,steam_purchase,received_for_free,written_during_early_access,primarily_steam_deck,timestamp_dev_responded,developer_response
0,188610035,"{'steamid': '76561198211799916', 'num_games_ow...",brazilian,[h1] Do lixo ao luxo. [/h1]\n\n[i] Cyberpunk 2...,1740342079,1740342196,True,135,3,0.904349,23,True,False,False,False,NaN,NaN
1,201379499,"{'steamid': '76561199486266183', 'num_games_ow...",brazilian,Ainda em dúvida se vale a pena jogar Cyberpunk...,1754244103,1754244254,True,97,11,0.888484,1,True,False,False,False,NaN,NaN
2,182786617,"{'steamid': '76561198262085799', 'num_games_ow...",brazilian,[h3] Cyberpunk 2077 é uma experiência de vida....,1734238952,1734239311,True,72,0,0.864477,1,True,False,False,False,NaN,NaN
3,197361390,"{'steamid': '76561197987117593', 'num_games_ow...",brazilian,Eu comprei Cyberpunk na pre venda em 2020 e po...,1750059222,1750059222,True,53,2,0.844979,0,True,False,False,False,NaN,NaN
4,180156366,"{'steamid': '76561198823290854', 'num_games_ow...",brazilian,Cyberpunk 2077 é um jogo excepcional que se de...,1732753942,1732753942,True,67,0,0.837374,8,True,False,False,False,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41422,200282463,"{'steamid': '76561199781047855', 'num_games_ow...",english,Very Nice,1753077136,1753077136,True,0,0,0.500000,0,True,False,False,False,NaN,NaN
41423,200281021,"{'steamid': '76561199497829608', 'num_games_ow...",english,great,1753074611,1753074611,True,0,0,0.500000,0,True,False,False,True,NaN,NaN
41424,200275037,"{'steamid': '76561199058300177', 'num_games_ow...",english,10/10,1753065378,1753065378,True,0,0,0.500000,0,True,False,False,False,NaN,NaN
41425,200270862,"{'steamid': '76561199005260491', 'num_games_ow...",english,"HAAAANK!\n\nHANK, DON'T ABBREVIATE CYBERPUNK.\...",1753059527,1753059546,True,0,0,0.500000,0,True,False,False,False,NaN,NaN


# Visão Geral e entedimento dos dados
- Verificação dos tipos das colunas, dados duplicados e dados nulos.
- Entendendo como os dados estão divididos
- Entendendo o que é cada coluna

**Após a análise desses três pontos, serão tomadas decisões para tratamento de dados: conversões de tipos, remoção de tabelas e pré processamento.**


In [26]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41427 entries, 0 to 41426
Data columns (total 17 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   recommendationid             41427 non-null  int64  
 1   author                       41427 non-null  object 
 2   language                     41427 non-null  object 
 3   review                       41409 non-null  object 
 4   timestamp_created            41427 non-null  int64  
 5   timestamp_updated            41427 non-null  int64  
 6   voted_up                     41427 non-null  bool   
 7   votes_up                     41427 non-null  int64  
 8   votes_funny                  41427 non-null  int64  
 9   weighted_vote_score          41427 non-null  float64
 10  comment_count                41427 non-null  int64  
 11  steam_purchase               41427 non-null  bool   
 12  received_for_free            41427 non-null  bool   
 13  written_during_e

-  Será necessário realizar algumas conversões principalmente para data que está em segundos, e a coluna de autor, que tem um dicionário com várias informações sobre o autor.

- Além disso, verificando as colunas nossa decisão inicial para exploração dos dados, será remover algumas colunas do dataset desnecessárias para análise


In [27]:
df.isnull().sum()

,0
recommendationid,0
author,0
language,0
review,18
timestamp_created,0
timestamp_updated,0
voted_up,0
votes_up,0
votes_funny,0
weighted_vote_score,0


Analisando agora os dados nulos, encontramos 18 comentários nulos e 41424 amostras que não tem comentários respondidos pelo desenvolvedores do jogo, ou seja é uma rara ocasião, no nosso dataset apenas 3 comentários foram respondidas.

**Nesse caso optamos por remover essas duas colunas, e as linhas onde os comentários são nulos.**

In [28]:
print(df.duplicated().sum())

0


Nenhum dado duplicado, como esperado, esse problema já foi tratado na coleta via API (usando recommendationid como chave única)

In [29]:
print(df['language'].value_counts())

pt_reviews = df['language'].value_counts()['brazilian']
en_reviews = df['language'].value_counts()['english']

print(f"Reviews in portuguese:{pt_reviews} ")
print(f"Reviews in english:{en_reviews} ")

language
english      34515
brazilian     6912
Name: count, dtype: int64
Reviews in portuguese:6912 
Reviews in english:34515 


Fizemos a requisição em duas linguagem principais, inglês e português, nosso dataset atual tem mais comentários em inglês do que em português. O dataset é desbanlanceado, mas como vamos comparar percepções entre essas duas linguagens vamos manter.

In [30]:
print(df['author'][1])

{'steamid': '76561199486266183', 'num_games_owned': 534, 'num_reviews': 6, 'playtime_forever': 9865, 'playtime_last_two_weeks': 0, 'playtime_at_review': 4919, 'deck_playtime_at_review': 5, 'last_played': 1754595467}


A coluna author, é uma coluna que contém um dicionário com diversas informações do autor da avaliação, será necessário tratamento



In [31]:
df.describe()

,recommendationid,timestamp_created,timestamp_updated,votes_up,votes_funny,weighted_vote_score,comment_count,timestamp_dev_responded
count,4.142700e+04,4.142700e+04,4.142700e+04,41427.000000,41427.000000,41427.000000,41427.000000,3.000000e+00
mean,1.931065e+08,1.745428e+09,1.745731e+09,1.182466,0.176310,0.501531,0.058078,1.630436e+09
std,8.579806e+06,9.297688e+06,9.291197e+06,24.557554,5.055527,0.026747,1.523091,1.100474e+06
min,8.191977e+07,1.607560e+09,1.607574e+09,0.000000,0.000000,0.276686,0.000000,1.629800e+09
25%,1.858309e+08,1.737174e+09,1.737526e+09,0.000000,0.000000,0.500000,0.000000,1.629800e+09
50%,1.936469e+08,1.745691e+09,1.746393e+09,0.000000,0.000000,0.500000,0.000000,1.629801e+09
75%,2.006962e+08,1.753471e+09,1.753742e+09,0.000000,0.000000,0.500000,0.000000,1.630754e+09
max,2.067418e+08,1.760477e+09,1.760477e+09,2236.000000,778.000000,0.968308,262.000000,1.631706e+09


# Tratamento de dados: conversão de tipos e remoção de colunas
- Após entedimento dos valores da coluna iremos realizar algumas conversões de tipagem necessária
- Remoção de algumas colunas desnecessárias para nossa análise.

Conversão das datas, de segundos para o formato legível

In [32]:

df['timestamp_created'] = pd.to_datetime(df['timestamp_created'], unit='s')
df['timestamp_updated'] = pd.to_datetime(df['timestamp_updated'], unit='s')
df

,recommendationid,author,language,review,timestamp_created,timestamp_updated,voted_up,votes_up,votes_funny,weighted_vote_score,comment_count,steam_purchase,received_for_free,written_during_early_access,primarily_steam_deck,timestamp_dev_responded,developer_response
0,188610035,"{'steamid': '76561198211799916', 'num_games_ow...",brazilian,[h1] Do lixo ao luxo. [/h1]\n\n[i] Cyberpunk 2...,2025-02-23 20:21:19,2025-02-23 20:23:16,True,135,3,0.904349,23,True,False,False,False,NaN,NaN
1,201379499,"{'steamid': '76561199486266183', 'num_games_ow...",brazilian,Ainda em dúvida se vale a pena jogar Cyberpunk...,2025-08-03 18:01:43,2025-08-03 18:04:14,True,97,11,0.888484,1,True,False,False,False,NaN,NaN
2,182786617,"{'steamid': '76561198262085799', 'num_games_ow...",brazilian,[h3] Cyberpunk 2077 é uma experiência de vida....,2024-12-15 05:02:32,2024-12-15 05:08:31,True,72,0,0.864477,1,True,False,False,False,NaN,NaN
3,197361390,"{'steamid': '76561197987117593', 'num_games_ow...",brazilian,Eu comprei Cyberpunk na pre venda em 2020 e po...,2025-06-16 07:33:42,2025-06-16 07:33:42,True,53,2,0.844979,0,True,False,False,False,NaN,NaN
4,180156366,"{'steamid': '76561198823290854', 'num_games_ow...",brazilian,Cyberpunk 2077 é um jogo excepcional que se de...,2024-11-28 00:32:22,2024-11-28 00:32:22,True,67,0,0.837374,8,True,False,False,False,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41422,200282463,"{'steamid': '76561199781047855', 'num_games_ow...",english,Very Nice,2025-07-21 05:52:16,2025-07-21 05:52:16,True,0,0,0.500000,0,True,False,False,False,NaN,NaN
41423,200281021,"{'steamid': '76561199497829608', 'num_games_ow...",english,great,2025-07-21 05:10:11,2025-07-21 05:10:11,True,0,0,0.500000,0,True,False,False,True,NaN,NaN
41424,200275037,"{'steamid': '76561199058300177', 'num_games_ow...",english,10/10,2025-07-21 02:36:18,2025-07-21 02:36:18,True,0,0,0.500000,0,True,False,False,False,NaN,NaN
41425,200270862,"{'steamid': '76561199005260491', 'num_games_ow...",english,"HAAAANK!\n\nHANK, DON'T ABBREVIATE CYBERPUNK.\...",2025-07-21 00:58:47,2025-07-21 00:59:06,True,0,0,0.500000,0,True,False,False,False,NaN,NaN


Extraindo features da coluna author, que vimos que era necessário tratamento.

In [33]:
import ast

df['author_dict'] = df['author'].apply(ast.literal_eval)

author_features = df['author_dict'].apply(pd.Series)

df['steamid'] = author_features['steamid']
df['num_games_owned'] = author_features['num_games_owned']
df['num_reviews'] = author_features['num_reviews']
df['playtime_forever'] = author_features['playtime_forever']
df['playtime_last_two_weeks'] = author_features['playtime_last_two_weeks']
df['playtime_at_review'] = author_features['playtime_at_review']
df['last_played'] = author_features['last_played']

df = df.drop(columns=['author', 'author_dict'])

print("\nFeatures extracds of 'author':")
print(author_features.columns)

df


Features extracds of 'author':
Index(['steamid', 'num_games_owned', 'num_reviews', 'playtime_forever',
       'playtime_last_two_weeks', 'playtime_at_review', 'last_played',
       'deck_playtime_at_review'],
      dtype='object')


,recommendationid,language,review,timestamp_created,timestamp_updated,voted_up,votes_up,votes_funny,weighted_vote_score,comment_count,...,primarily_steam_deck,timestamp_dev_responded,developer_response,steamid,num_games_owned,num_reviews,playtime_forever,playtime_last_two_weeks,playtime_at_review,last_played
0,188610035,brazilian,[h1] Do lixo ao luxo. [/h1]\n\n[i] Cyberpunk 2...,2025-02-23 20:21:19,2025-02-23 20:23:16,True,135,3,0.904349,23,...,False,NaN,NaN,76561198211799916,506,130,8964,0,8964,1738185053
1,201379499,brazilian,Ainda em dúvida se vale a pena jogar Cyberpunk...,2025-08-03 18:01:43,2025-08-03 18:04:14,True,97,11,0.888484,1,...,False,NaN,NaN,76561199486266183,534,6,9865,0,4919,1754595467
2,182786617,brazilian,[h3] Cyberpunk 2077 é uma experiência de vida....,2024-12-15 05:02:32,2024-12-15 05:08:31,True,72,0,0.864477,1,...,False,NaN,NaN,76561198262085799,82,32,4898,0,4574,1753038202
3,197361390,brazilian,Eu comprei Cyberpunk na pre venda em 2020 e po...,2025-06-16 07:33:42,2025-06-16 07:33:42,True,53,2,0.844979,0,...,False,NaN,NaN,76561197987117593,336,42,15237,0,12634,1754580367
4,180156366,brazilian,Cyberpunk 2077 é um jogo excepcional que se de...,2024-11-28 00:32:22,2024-11-28 00:32:22,True,67,0,0.837374,8,...,False,NaN,NaN,76561198823290854,0,4,16573,0,6887,1757792681
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41422,200282463,english,Very Nice,2025-07-21 05:52:16,2025-07-21 05:52:16,True,0,0,0.500000,0,...,False,NaN,NaN,76561199781047855,0,1,3714,0,3594,1753096132
41423,200281021,english,great,2025-07-21 05:10:11,2025-07-21 05:10:11,True,0,0,0.500000,0,...,True,NaN,NaN,76561199497829608,0,1,7341,29,6999,1759291212
41424,200275037,english,10/10,2025-07-21 02:36:18,2025-07-21 02:36:18,True,0,0,0.500000,0,...,False,NaN,NaN,76561199058300177,0,1,4821,52,1945,1759469032
41425,200270862,english,"HAAAANK!\n\nHANK, DON'T ABBREVIATE CYBERPUNK.\...",2025-07-21 00:58:47,2025-07-21 00:59:06,True,0,0,0.500000,0,...,False,NaN,NaN,76561199005260491,230,14,10968,32,10391,1759282695


# Visão geral pós tratamento de tipos

In [34]:
df.describe()

,recommendationid,timestamp_created,timestamp_updated,votes_up,votes_funny,weighted_vote_score,comment_count,timestamp_dev_responded,num_games_owned,num_reviews,playtime_forever,playtime_last_two_weeks,playtime_at_review,last_played
count,4.142700e+04,41427,41427,41427.000000,41427.000000,41427.000000,41427.000000,3.000000e+00,41427.000000,41427.000000,41427.000000,41427.000000,41427.000000,4.142700e+04
mean,1.931065e+08,2025-04-23 17:11:05.746928384,2025-04-27 05:14:13.952832768,1.182466,0.176310,0.501531,0.058078,1.630436e+09,77.641055,14.417409,8661.111570,160.027687,6253.079610,1.751322e+09
min,8.191977e+07,2020-12-10 00:34:32,2020-12-10 04:14:17,0.000000,0.000000,0.276686,0.000000,1.629800e+09,0.000000,1.000000,5.000000,0.000000,5.000000,1.607609e+09
25%,1.858309e+08,2025-01-18 04:19:40.500000,2025-01-22 06:10:22.500000,0.000000,0.000000,0.500000,0.000000,1.629800e+09,0.000000,3.000000,3152.000000,0.000000,1586.000000,1.747591e+09
50%,1.936469e+08,2025-04-26 18:11:49,2025-05-04 21:15:53,0.000000,0.000000,0.500000,0.000000,1.629801e+09,0.000000,7.000000,5892.000000,0.000000,3767.000000,1.755188e+09
75%,2.006962e+08,2025-07-25 19:21:12.500000,2025-07-28 22:35:50.500000,0.000000,0.000000,0.500000,0.000000,1.630754e+09,83.000000,16.000000,10175.000000,0.000000,7485.000000,1.758909e+09
max,2.067418e+08,2025-10-14 21:27:56,2025-10-14 21:27:56,2236.000000,778.000000,0.968308,262.000000,1.631706e+09,27994.000000,5264.000000,636446.000000,15633.000000,530186.000000,1.760479e+09
std,8.579806e+06,NaN,NaN,24.557554,5.055527,0.026747,1.523091,1.100474e+06,271.344024,36.594096,11939.232055,645.738860,10077.324749,1.208006e+07


Aqui nosso interesse foi analisar a faixa de tempo que temos nos dados, então temos dados em uma faixa temporal de 5 anos, desde 10/12/2020 até 14/10/2025. Data de lançamento até a data de processamento aquisição dos dados.

In [35]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41427 entries, 0 to 41426
Data columns (total 23 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   recommendationid             41427 non-null  int64         
 1   language                     41427 non-null  object        
 2   review                       41409 non-null  object        
 3   timestamp_created            41427 non-null  datetime64[ns]
 4   timestamp_updated            41427 non-null  datetime64[ns]
 5   voted_up                     41427 non-null  bool          
 6   votes_up                     41427 non-null  int64         
 7   votes_funny                  41427 non-null  int64         
 8   weighted_vote_score          41427 non-null  float64       
 9   comment_count                41427 non-null  int64         
 10  steam_purchase               41427 non-null  bool          
 11  received_for_free            41427 non-nu

Após conversão de tipagem e extração de caracteristicas da coluna author, temos essas 22 tabelas.

Algumas tabelas não necessárias para nossa análise EDA, pois não fazem muito sentido:
- recommendationid: ID técnico sem valor analítico
- timestamp_dev_responded: 99.99% missing
- developer_response: 99.99% missing
- written_during_early_access: Sempre False
- primarily_steam_deck: Não relevante para análise
- timestamp_updated: timestamp_created já é suficiente
- steamid: ID do usuário (privacidade)
- playtime_last_two_weeks: Dado momentâneo
- last_played: Não relevante

Além de remover as amostras de comentários nulos

então construindo o dataset com os dados limpos...

In [36]:
df.dropna(subset=['review'], inplace=True)


In [37]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 41409 entries, 0 to 41426
Data columns (total 23 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   recommendationid             41409 non-null  int64         
 1   language                     41409 non-null  object        
 2   review                       41409 non-null  object        
 3   timestamp_created            41409 non-null  datetime64[ns]
 4   timestamp_updated            41409 non-null  datetime64[ns]
 5   voted_up                     41409 non-null  bool          
 6   votes_up                     41409 non-null  int64         
 7   votes_funny                  41409 non-null  int64         
 8   weighted_vote_score          41409 non-null  float64       
 9   comment_count                41409 non-null  int64         
 10  steam_purchase               41409 non-null  bool          
 11  received_for_free            41409 non-null  b

In [38]:
df =  df.drop(columns=['recommendationid', 'written_during_early_access', 'timestamp_updated', 'steamid', 'primarily_steam_deck', 'timestamp_dev_responded', 'developer_response', 'playtime_last_two_weeks', 'last_played'])
df

,language,review,timestamp_created,voted_up,votes_up,votes_funny,weighted_vote_score,comment_count,steam_purchase,received_for_free,num_games_owned,num_reviews,playtime_forever,playtime_at_review
0,brazilian,[h1] Do lixo ao luxo. [/h1]\n\n[i] Cyberpunk 2...,2025-02-23 20:21:19,True,135,3,0.904349,23,True,False,506,130,8964,8964
1,brazilian,Ainda em dúvida se vale a pena jogar Cyberpunk...,2025-08-03 18:01:43,True,97,11,0.888484,1,True,False,534,6,9865,4919
2,brazilian,[h3] Cyberpunk 2077 é uma experiência de vida....,2024-12-15 05:02:32,True,72,0,0.864477,1,True,False,82,32,4898,4574
3,brazilian,Eu comprei Cyberpunk na pre venda em 2020 e po...,2025-06-16 07:33:42,True,53,2,0.844979,0,True,False,336,42,15237,12634
4,brazilian,Cyberpunk 2077 é um jogo excepcional que se de...,2024-11-28 00:32:22,True,67,0,0.837374,8,True,False,0,4,16573,6887
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41422,english,Very Nice,2025-07-21 05:52:16,True,0,0,0.500000,0,True,False,0,1,3714,3594
41423,english,great,2025-07-21 05:10:11,True,0,0,0.500000,0,True,False,0,1,7341,6999
41424,english,10/10,2025-07-21 02:36:18,True,0,0,0.500000,0,True,False,0,1,4821,1945
41425,english,"HAAAANK!\n\nHANK, DON'T ABBREVIATE CYBERPUNK.\...",2025-07-21 00:58:47,True,0,0,0.500000,0,True,False,230,14,10968,10391


In [39]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 41409 entries, 0 to 41426
Data columns (total 14 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   language             41409 non-null  object        
 1   review               41409 non-null  object        
 2   timestamp_created    41409 non-null  datetime64[ns]
 3   voted_up             41409 non-null  bool          
 4   votes_up             41409 non-null  int64         
 5   votes_funny          41409 non-null  int64         
 6   weighted_vote_score  41409 non-null  float64       
 7   comment_count        41409 non-null  int64         
 8   steam_purchase       41409 non-null  bool          
 9   received_for_free    41409 non-null  bool          
 10  num_games_owned      41409 non-null  int64         
 11  num_reviews          41409 non-null  int64         
 12  playtime_forever     41409 non-null  int64         
 13  playtime_at_review   41409 non-null 

In [40]:
df.isnull().sum()

,0
language,0
review,0
timestamp_created,0
voted_up,0
votes_up,0
votes_funny,0
weighted_vote_score,0
comment_count,0
steam_purchase,0
received_for_free,0


In [41]:
rename_map = {
    'review': 'review_text',
    'timestamp_created': 'created_at',
    'voted_up': 'is_recommended',
    'votes_up': 'helpful_votes',
    'votes_funny': 'funny_votes',
    'weighted_vote_score': 'vote_score',
    'comment_count': 'num_comments',
    'steam_purchase': 'is_steam_purchase',
    'received_for_free': 'is_free',
    'playtime_forever': 'total_playtime_min',
    'playtime_at_review': 'playtime_at_review_min'
}

df = df.rename(columns=rename_map)
df

,language,review_text,created_at,is_recommended,helpful_votes,funny_votes,vote_score,num_comments,is_steam_purchase,is_free,num_games_owned,num_reviews,total_playtime_min,playtime_at_review_min
0,brazilian,[h1] Do lixo ao luxo. [/h1]\n\n[i] Cyberpunk 2...,2025-02-23 20:21:19,True,135,3,0.904349,23,True,False,506,130,8964,8964
1,brazilian,Ainda em dúvida se vale a pena jogar Cyberpunk...,2025-08-03 18:01:43,True,97,11,0.888484,1,True,False,534,6,9865,4919
2,brazilian,[h3] Cyberpunk 2077 é uma experiência de vida....,2024-12-15 05:02:32,True,72,0,0.864477,1,True,False,82,32,4898,4574
3,brazilian,Eu comprei Cyberpunk na pre venda em 2020 e po...,2025-06-16 07:33:42,True,53,2,0.844979,0,True,False,336,42,15237,12634
4,brazilian,Cyberpunk 2077 é um jogo excepcional que se de...,2024-11-28 00:32:22,True,67,0,0.837374,8,True,False,0,4,16573,6887
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41422,english,Very Nice,2025-07-21 05:52:16,True,0,0,0.500000,0,True,False,0,1,3714,3594
41423,english,great,2025-07-21 05:10:11,True,0,0,0.500000,0,True,False,0,1,7341,6999
41424,english,10/10,2025-07-21 02:36:18,True,0,0,0.500000,0,True,False,0,1,4821,1945
41425,english,"HAAAANK!\n\nHANK, DON'T ABBREVIATE CYBERPUNK.\...",2025-07-21 00:58:47,True,0,0,0.500000,0,True,False,230,14,10968,10391


- Removidas as colunas consideradas desnecessárias
- Tipos de dados bem convertidos e legivéis
- Nenhum comentário nulo para analisar percepções e sentimentos
- Padronizar nomes de colunas

In [42]:
df.to_csv('cyberpunk_steam_reviews_clean.csv', index=False)